# THE BATTLE OF NEIGHBORHOODS PROJECT

##### Applied Data Science Capstone Project by Yukun Zhao

## Table of Contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

In this project, we will try to find an optimal location for opening a Chinese restaurant in York Region, Ontario, Canada. 

We will look at population and location data to find a neighborhood in York Region that is suitable for opening the Chinese restaurant. 

In addition, we should also look into the competitions in the neighborhood. This could be be done using the location data to find out if there are opportunities to provide something that is not yet available.

## Data <a name="data"></a>

Factors that will influence our decision are:
* number of Chinese households in the city,
* number of existing restaurants in the neighborhood (Chinese or other asian restaurants),
* number of parks, local event venues, malls.

Following data sources will be needed to obtain the required information:
* 2016 Census of Canada from Statistics Canada,
* malls and plazas in the city of interest,
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare Places API**.

## Methodology <a name="methodology"></a>

In this project, we will start with finding the city in York Region that has the largest Chinese community.

Next we will look at the malls and plazas in the city of interest. We will find the location and type (category) of every restaurant within 1km radius from the center of each place. We will also identify other venues, such as parks, gyms and dessert in vicinity.

Third step in our analysis will be exploration of venues. We will find the venue categories, and use clustering to group the malls and plazas based these venue categories. These clusters should be a starting point for final 'street level' exploration and search for optimal venue location by our client.

In the final step, we will focus on those promising areas. We will look into the type of Chinese cuisines served in these areas. This should give our client some ideas what food should be served.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 101)
pd.set_option('display.max_columns', None)

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import AgglomerativeClustering

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Find the city with the largest Chinese community

In [2]:
# The code was removed by Watson Studio for sharing.

,OBJECTID,DAUID,CSDUID,CSDNAME,TOT_VISIBLE_MINORITY_PVT_HH,TOT_VISIBLE_MINORITY_POP,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE,VISIBLE_MINORITY_NIE,MULTIPLE_VISIBLE_MINORITY,NOT_VISIBLE_MINORITY,SHAPE.STArea(),SHAPE.STLength()
0,1,35191319,3519028,Vaughan,1055.0,440.0,55.0,130.0,15.0,70.0,0.0,0.0,10.0,25.0,125.0,0.0,10.0,0.0,615.0,1.657006e+05,1881.204066
1,2,35191314,3519028,Vaughan,1230.0,490.0,95.0,210.0,0.0,35.0,0.0,0.0,0.0,45.0,45.0,0.0,15.0,40.0,740.0,6.353679e+05,3568.832137
2,3,35191037,3519028,Vaughan,6330.0,2565.0,925.0,505.0,225.0,335.0,105.0,120.0,85.0,135.0,45.0,20.0,45.0,25.0,3765.0,1.405759e+06,5055.080921
3,4,35190824,3519028,Vaughan,2150.0,455.0,280.0,70.0,35.0,15.0,0.0,0.0,20.0,10.0,0.0,0.0,25.0,10.0,1700.0,6.461407e+05,4272.214232
4,5,35190825,3519028,Vaughan,565.0,100.0,70.0,0.0,0.0,0.0,15.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,465.0,2.537939e+05,2756.348141


#### clean the census data

In [3]:
# drop the irrelevant columns
column_names = census2016.columns.values
drop_index = [0, 1, 2, 4, 5, 16, 17, 18, 19, 20]
drop_columns = []
for idx in drop_index:
    drop_columns.append(column_names[idx])
census2016.drop(labels = drop_columns, axis = 1, inplace = True)
census2016.head()

,CSDNAME,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
0,Vaughan,55.0,130.0,15.0,70.0,0.0,0.0,10.0,25.0,125.0,0.0
1,Vaughan,95.0,210.0,0.0,35.0,0.0,0.0,0.0,45.0,45.0,0.0
2,Vaughan,925.0,505.0,225.0,335.0,105.0,120.0,85.0,135.0,45.0,20.0
3,Vaughan,280.0,70.0,35.0,15.0,0.0,0.0,20.0,10.0,0.0,0.0
4,Vaughan,70.0,0.0,0.0,0.0,15.0,0.0,10.0,10.0,0.0,0.0


In [4]:
# change CSDNAME to City
census2016.rename(columns = {'CSDNAME': 'CITY'}, inplace = True)
census2016.head()

,CITY,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
0,Vaughan,55.0,130.0,15.0,70.0,0.0,0.0,10.0,25.0,125.0,0.0
1,Vaughan,95.0,210.0,0.0,35.0,0.0,0.0,0.0,45.0,45.0,0.0
2,Vaughan,925.0,505.0,225.0,335.0,105.0,120.0,85.0,135.0,45.0,20.0
3,Vaughan,280.0,70.0,35.0,15.0,0.0,0.0,20.0,10.0,0.0,0.0
4,Vaughan,70.0,0.0,0.0,0.0,15.0,0.0,10.0,10.0,0.0,0.0


In [5]:
# group the data by city
census2016_sum = census2016.groupby(by = 'CITY').sum()
census2016_sum

,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
CITY,,,,,,,,,,
Aurora,1840.0,5575.0,1200.0,935.0,670.0,575.0,300.0,1870.0,595.0,195.0
Chippewas of Georgina Island First Nation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
East Gwillimbury,470.0,690.0,335.0,165.0,100.0,85.0,70.0,255.0,90.0,60.0
Georgina,685.0,670.0,695.0,335.0,375.0,100.0,130.0,185.0,95.0,70.0
King,930.0,445.0,395.0,160.0,170.0,125.0,140.0,320.0,160.0,70.0
Markham,58255.0,147710.0,9700.0,8945.0,1730.0,3240.0,2460.0,7940.0,4350.0,1015.0
Newmarket,3265.0,5565.0,2295.0,1835.0,1225.0,660.0,2275.0,1970.0,695.0,325.0
Richmond Hill,14910.0,56970.0,3850.0,3790.0,1580.0,3565.0,1415.0,20120.0,5485.0,625.0
Vaughan,30655.0,20790.0,8345.0,8645.0,7415.0,4325.0,6840.0,8650.0,5360.0,300.0


In [6]:
# sort by number of Chinese
census2016_sum.sort_values(by = ['CHINESE'], axis = 0, ascending = False, inplace = True)
census2016_sum

,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
CITY,,,,,,,,,,
Markham,58255.0,147710.0,9700.0,8945.0,1730.0,3240.0,2460.0,7940.0,4350.0,1015.0
Richmond Hill,14910.0,56970.0,3850.0,3790.0,1580.0,3565.0,1415.0,20120.0,5485.0,625.0
Vaughan,30655.0,20790.0,8345.0,8645.0,7415.0,4325.0,6840.0,8650.0,5360.0,300.0
Whitchurch-Stouffville,5625.0,5965.0,995.0,1135.0,330.0,435.0,295.0,335.0,190.0,150.0
Aurora,1840.0,5575.0,1200.0,935.0,670.0,575.0,300.0,1870.0,595.0,195.0
Newmarket,3265.0,5565.0,2295.0,1835.0,1225.0,660.0,2275.0,1970.0,695.0,325.0
East Gwillimbury,470.0,690.0,335.0,165.0,100.0,85.0,70.0,255.0,90.0,60.0
Georgina,685.0,670.0,695.0,335.0,375.0,100.0,130.0,185.0,95.0,70.0
King,930.0,445.0,395.0,160.0,170.0,125.0,140.0,320.0,160.0,70.0


We now know that Markham has the largest Chinese community in York Region. Let us sort the number in Markham to see what the numbers are for each visible minority group.

In [7]:
# sort the columns by the values in Markham
census2016_sum.sort_values(by = ['Markham'], axis = 1, ascending = False, inplace = True)
census2016_sum

,CHINESE,SOUTH_ASIAN,BLACK,FILIPINO,WEST_ASIAN,KOREAN,ARAB,SOUTHEAST_ASIAN,LATIN_AMERICAN,JAPANESE
CITY,,,,,,,,,,
Markham,147710.0,58255.0,9700.0,8945.0,7940.0,4350.0,3240.0,2460.0,1730.0,1015.0
Richmond Hill,56970.0,14910.0,3850.0,3790.0,20120.0,5485.0,3565.0,1415.0,1580.0,625.0
Vaughan,20790.0,30655.0,8345.0,8645.0,8650.0,5360.0,4325.0,6840.0,7415.0,300.0
Whitchurch-Stouffville,5965.0,5625.0,995.0,1135.0,335.0,190.0,435.0,295.0,330.0,150.0
Aurora,5575.0,1840.0,1200.0,935.0,1870.0,595.0,575.0,300.0,670.0,195.0
Newmarket,5565.0,3265.0,2295.0,1835.0,1970.0,695.0,660.0,2275.0,1225.0,325.0
East Gwillimbury,690.0,470.0,335.0,165.0,255.0,90.0,85.0,70.0,100.0,60.0
Georgina,670.0,685.0,695.0,335.0,185.0,95.0,100.0,130.0,375.0,70.0
King,445.0,930.0,395.0,160.0,320.0,160.0,125.0,140.0,170.0,70.0


We now know that Markham has the largest Chinese community. The numbers for other communities are useful when we select the food that we will serve. We should now look at the neighborhoods in Markham.

### 2. Explore Markham

In [8]:
# load malls and plazas in York Region
body2 = client_60fde9a69b664de08697ac05914adba4.get_object(Bucket='capstone-donotdelete-pr-rtwgqykbmttzjy',Key='mall_plaza.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

mall_plaza = pd.read_excel(body2)
mall_plaza.head()

,CITY,NAME,ADDRESS
0,Markham,Alderland Center,"7102-7158 Warden Ave, Markham, ON"
1,Markham,CF Markville,"5000 Hwy 7, Markham, ON"
2,Markham,First Markham Place,"3255 Highway 7 East, Markham, ON"
3,Markham,J-Town,"160 Steeles Ave E, Markham, ON"
4,Markham,King Square Shopping Mall,"9390 Woodbine Ave, Markham, ON"


In [9]:
# use geopy to get the geographic coordinates for these places
mall_plaza_latitude = []
mall_plaza_longitude = []
geolocator = Nominatim(user_agent = "markham_explorer")

for idx in mall_plaza.index.values:
    # use the address first
    address = mall_plaza.loc[idx, 'ADDRESS']
    location = geolocator.geocode(address)
    if location == None:
        # use name
        address = mall_plaza.loc[idx, 'NAME']
        location = geolocator.geocode(address)
        mall_plaza_latitude.append(location.latitude)
        mall_plaza_longitude.append(location.longitude)
    else:
        mall_plaza_latitude.append(location.latitude)
        mall_plaza_longitude.append(location.longitude)

# add the coordinates to the dataframe
mall_plaza['LATITUDE'] = mall_plaza_latitude
mall_plaza['LONGITUDE'] = mall_plaza_longitude
mall_plaza

,CITY,NAME,ADDRESS,LATITUDE,LONGITUDE
0,Markham,Alderland Center,"7102-7158 Warden Ave, Markham, ON",43.824235,-79.326603
1,Markham,CF Markville,"5000 Hwy 7, Markham, ON",43.865694,-79.291642
2,Markham,First Markham Place,"3255 Highway 7 East, Markham, ON",43.851747,-79.348356
3,Markham,J-Town,"160 Steeles Ave E, Markham, ON",43.801437,-79.405085
4,Markham,King Square Shopping Mall,"9390 Woodbine Ave, Markham, ON",43.872314,-79.365975
5,Markham,Langham Square,"8339 Kennedy Rd, Markham, ON",43.857400,-79.303767
6,Markham,Markham Town Square,"8601 Warden Ave, Markham, ON",43.857494,-79.331529
7,Markham,Market Village,"4390 Steeles Avenue East, Markham, ON",43.824568,-79.303737
8,Markham,Metro Square,"3636 Steeles Ave E, Markham, ON",43.817998,-79.331931
9,Markham,New Kennedy Square,"8360 Kennedy Rd, Markham, ON",43.859604,-79.302949


In [10]:
# see Markham neighborhoods on a map
address = 'Markham, ON'
geolocator = Nominatim(user_agent = "markham_explorer")
location = geolocator.geocode(address)
latitude_markham = location.latitude
longitude_markham = location.longitude

# create map object
map_markham = folium.Map(location=[latitude_markham, longitude_markham], zoom_start=11)

# add markers to map
for lat, lng, label in zip(mall_plaza['LATITUDE'], mall_plaza['LONGITUDE'], mall_plaza['NAME']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_markham)  
    
map_markham

We can tell from the map that all the plazas and malls have parking lots, and they are close to some residential areas.

#### find the venues around these malls or plazas from Foursquare

In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT = 100, radius = 1000):
    
    '''
    
    names: mall/plaza names,
    latitudes, longitudes: geospatial coordinates of the neighborhoods,
    Limit: maximum number of venues returned,
    radius: search radius in meters.
    
    '''
    
    venues_list = []
    count = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])
        count += 1

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Mall/plaza', 
                  'Mall/plaza Latitude', 
                  'Mall/plaza Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue ID']
    
    #show number of neighborhood explored
    print('\n'+ " ".join([str(count), 'neighborhoods explored.']))
    
    return(nearby_venues)

In [13]:
markham_venues = getNearbyVenues(names = mall_plaza['NAME'],
                                   latitudes = mall_plaza['LATITUDE'],
                                   longitudes = mall_plaza['LONGITUDE'],
                                  LIMIT = 150, radius = 1000
                                )
print(markham_venues.shape)
markham_venues.head()

Alderland Center
CF Markville
First Markham Place
J-Town
King Square Shopping Mall
Langham Square
Markham Town Square
Market Village
Metro Square
New Kennedy Square
Pacific Mall
Peachtree Center
The Shops on Steeles and 404
Thornhill Square Shopping Center

14 neighborhoods explored.
(627, 8)


,Mall/plaza,Mall/plaza Latitude,Mall/plaza Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Alderland Center,43.824235,-79.326603,T&T Supermarket 大統華超級市場,43.821925,-79.326174,Supermarket,4af0b925f964a52092de21e3
1,Alderland Center,43.824235,-79.326603,Monte Carlo Inn & Suites Downtown Markham,43.828094,-79.325874,Hotel,4e248475d16474063224c3b0
2,Alderland Center,43.824235,-79.326603,Chatime,43.824342,-79.326062,Bubble Tea Shop,526176ea11d2a41c253a55e6
3,Alderland Center,43.824235,-79.326603,Yin Ji Chang Fen 銀記腸粉店,43.820277,-79.325142,Chinese Restaurant,58c1e6e801f4336e21f31b50
4,Alderland Center,43.824235,-79.326603,Asian Legend 味香村,43.818846,-79.331058,Chinese Restaurant,4babe7aaf964a520efd33ae3


#### analyze the venues

In [14]:
# check numbers of venues returned for each neighborhood
markham_venues_grouped = markham_venues.groupby('Mall/plaza', sort = False, as_index = False)[['Venue']].count()
markham_venues_grouped

,Mall/plaza,Venue
0,Alderland Center,48
1,CF Markville,66
2,First Markham Place,53
3,J-Town,10
4,King Square Shopping Mall,23
5,Langham Square,43
6,Markham Town Square,48
7,Market Village,59
8,Metro Square,66
9,New Kennedy Square,46


In [15]:
# get the number of unique venue categories
print('There are {} unique venue categories.'.format(len(markham_venues['Venue Category'].unique())))
markham_venues['Venue Category'].unique()

There are 129 unique venue categories.


array(['Supermarket', 'Hotel', 'Bubble Tea Shop', 'Chinese Restaurant',
       'Coffee Shop', 'Dessert Shop', 'Szechuan Restaurant', 'Tea Room',
       'Grocery Store', 'Ice Cream Shop', 'Sporting Goods Shop',
       'Asian Restaurant', 'Korean Restaurant', 'Bank', 'Sandwich Place',
       'Hong Kong Restaurant', 'Dim Sum Restaurant',
       'Xinjiang Restaurant', 'Gas Station', 'Dumpling Restaurant',
       'Pharmacy', 'Ramen Restaurant', 'Fast Food Restaurant',
       'Pizza Place', 'Convenience Store', 'Hotpot Restaurant',
       'Noodle House', 'Shoe Store', 'Furniture / Home Store', 'Food',
       'Bakery', 'Shipping Store', 'Baseball Field', 'Diner',
       'Bus Station', 'Gym / Fitness Center', 'Sushi Restaurant',
       'Yoga Studio', 'Cosmetics Shop', 'Toy / Game Store',
       'New American Restaurant', 'Seafood Restaurant', 'Chocolate Shop',
       'Clothing Store', 'Health Food Store', 'Shopping Mall',
       'Food Court', 'Burger Joint', 'Restaurant', 'Community Center',
 

Looking at these categories, we are going to drop any venues that does not affect our decision. This includes:

In [16]:
drop_list = ['Bank', 'Gas Station', 'Pharmacy', 'Rental Service', 'Discount Store', 'Shipping Store', 'Intersection', 'Automotive Shop', 'Dry Cleaner',\
            'Candy Store', 'Convenience Store', 'Deli ', 'Distribution Center', 'Auto Dealership', 'Bus Station', 'Bus Stop', 'Road', 'Thrift']

for category in drop_list:
    # get the index in the dataframe that will be dropped
    markham_venues.drop(index = np.where(markham_venues['Venue Category'].str.contains(category, case = False))[0].tolist(), inplace = True)
    markham_venues.reset_index(drop = True, inplace = True)

print('There are {} unique venue categories.'.format(len(markham_venues['Venue Category'].unique())))
markham_venues['Venue Category'].unique()

There are 112 unique venue categories.


array(['Supermarket', 'Hotel', 'Bubble Tea Shop', 'Chinese Restaurant',
       'Coffee Shop', 'Dessert Shop', 'Szechuan Restaurant', 'Tea Room',
       'Grocery Store', 'Ice Cream Shop', 'Sporting Goods Shop',
       'Asian Restaurant', 'Korean Restaurant', 'Sandwich Place',
       'Hong Kong Restaurant', 'Dim Sum Restaurant',
       'Xinjiang Restaurant', 'Dumpling Restaurant', 'Ramen Restaurant',
       'Fast Food Restaurant', 'Pizza Place', 'Hotpot Restaurant',
       'Noodle House', 'Shoe Store', 'Furniture / Home Store', 'Food',
       'Bakery', 'Baseball Field', 'Diner', 'Gym / Fitness Center',
       'Sushi Restaurant', 'Yoga Studio', 'Cosmetics Shop',
       'Toy / Game Store', 'New American Restaurant',
       'Seafood Restaurant', 'Chocolate Shop', 'Clothing Store',
       'Health Food Store', 'Shopping Mall', 'Food Court', 'Burger Joint',
       'Restaurant', 'Community Center', 'Cantonese Restaurant',
       'Beer Store', 'Electronics Store', 'Greek Restaurant',
       'Big

We do not need to distinguish between different types of Chinese restaurants at this stage. Rename all types that will affect our decision as "Chinese Restaurant".

In [17]:
from copy import deepcopy
markham_venues_mod = deepcopy(markham_venues)

# list of different Chinese restaurant
chinese_type = ['Chinese Restaurant', 'Szechuan Restaurant', 'Asian Restaurant', 'Hong Kong Restaurant', 'Dim Sum Restaurant', 'Xinjiang Restaurant',\
                'Dumpling Restaurant', 'Hotpot Restaurant', 'Noodle House', 'Cantonese Restaurant', 'Shanghai Restaurant', 'Taiwanese Restaurant', \
               'BBQ Joint', 'Cha Chaan Teng']
for restaurant in chinese_type:
    markham_venues_mod.loc[markham_venues_mod['Venue Category'].str.contains(restaurant, case = False), 'Venue Category'] = 'Chinese Restaurant'
print('There are {} unique venue categories.'.format(len(markham_venues['Venue Category'].unique())))
markham_venues_mod['Venue Category'].unique()

There are 112 unique venue categories.


array(['Supermarket', 'Hotel', 'Bubble Tea Shop', 'Chinese Restaurant',
       'Coffee Shop', 'Dessert Shop', 'Tea Room', 'Grocery Store',
       'Ice Cream Shop', 'Sporting Goods Shop', 'Korean Restaurant',
       'Sandwich Place', 'Ramen Restaurant', 'Fast Food Restaurant',
       'Pizza Place', 'Shoe Store', 'Furniture / Home Store', 'Food',
       'Bakery', 'Baseball Field', 'Diner', 'Gym / Fitness Center',
       'Sushi Restaurant', 'Yoga Studio', 'Cosmetics Shop',
       'Toy / Game Store', 'New American Restaurant',
       'Seafood Restaurant', 'Chocolate Shop', 'Clothing Store',
       'Health Food Store', 'Shopping Mall', 'Food Court', 'Burger Joint',
       'Restaurant', 'Community Center', 'Beer Store',
       'Electronics Store', 'Greek Restaurant', 'Big Box Store',
       'Music Venue', 'Jewelry Store', 'Health & Beauty Service',
       'Thai Restaurant', 'Gym', 'Luggage Store', 'Department Store',
       'Kids Store', 'Spa', 'Halal Restaurant', 'Bookstore',
       'Climbi

In [18]:
markham_venues_mod_grouped = markham_venues_mod.groupby('Mall/plaza', sort = False, as_index = False)[['Venue']].count()
markham_venues_mod_grouped

,Mall/plaza,Venue
0,Alderland Center,42
1,CF Markville,60
2,First Markham Place,52
3,J-Town,7
4,King Square Shopping Mall,17
5,Langham Square,36
6,Markham Town Square,43
7,Market Village,56
8,Metro Square,58
9,New Kennedy Square,40


J-Town has only 7 venues. Let us see if we should drop it.

In [19]:
markham_venues_mod[markham_venues_mod['Mall/plaza'] == 'J-Town']

,Mall/plaza,Mall/plaza Latitude,Mall/plaza Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
154,J-Town,43.801437,-79.405085,Cayne's Super Housewares,43.805153,-79.412594,Furniture / Home Store,4ae48161f964a520119b21e3
155,J-Town,43.801437,-79.405085,Ambrosia Natural Foods,43.803955,-79.415948,Health Food Store,4ae48904f964a520659b21e3
156,J-Town,43.801437,-79.405085,Dear Born baby,43.804398,-79.415351,Kids Store,4bf167a0c9ced13ab61681c5
157,J-Town,43.801437,-79.405085,Grandview Park,43.804632,-79.411041,Baseball Field,4be34de421d5a593b2cc1811
158,J-Town,43.801437,-79.405085,Cakes by Robert,43.806213,-79.410921,Dessert Shop,4c3634d02c8020a1faec8600
159,J-Town,43.801437,-79.405085,Sticker & More Warehouse Store,43.804855,-79.412684,Arts & Crafts Store,506f3954e4b07fcb1f8c3e82
160,J-Town,43.801437,-79.405085,Evex,43.803899,-79.415773,Luggage Store,4be9823d6295c9b64b458508


There is no restaurant in this area. Let us drop it.

In [20]:
markham_venues_mod.drop(index = markham_venues_mod[markham_venues_mod['Mall/plaza'] == 'J-Town'].index, inplace = True)
markham_venues_mod.reset_index(drop = True, inplace = True)

In [21]:
# one hot encoding for the venues in Markham
markham_onehot = pd.get_dummies(markham_venues_mod[['Venue Category']], columns = ['Venue Category'], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
markham_onehot['Mall/plaza'] = markham_venues_mod['Mall/plaza'] 

# move neighborhood column to the first column
col = markham_onehot.columns.tolist()
markham_onehot = markham_onehot[col[-1:] + col[:-1]]

markham_onehot.head()

,Mall/plaza,Airport,American Restaurant,Arts & Crafts Store,Bagel Shop,Bakery,Bar,Baseball Field,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Community Center,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Flower Shop,Food,Food & Drink Shop,Food Court,Furniture / Home Store,Gastropub,German Restaurant,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Health Food Store,Hobby Shop,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Music Venue,New American Restaurant,Night Market,Optical Shop,Park,Pet Store,Pizza Place,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Yoga Studio
0,Alderland Center,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,Alderland Center,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alderland Center,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alderland Center,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alderland Center,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### let's find out the occurance of each venue category

In [22]:
markham_grouped = markham_onehot.groupby('Mall/plaza').mean().reset_index()
markham_grouped

,Mall/plaza,Airport,American Restaurant,Arts & Crafts Store,Bagel Shop,Bakery,Bar,Baseball Field,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Community Center,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Flower Shop,Food,Food & Drink Shop,Food Court,Furniture / Home Store,Gastropub,German Restaurant,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Health Food Store,Hobby Shop,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Music Venue,New American Restaurant,Night Market,Optical Shop,Park,Pet Store,Pizza Place,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Yoga Studio
0,Alderland Center,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.02381,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.380952,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.023810,0.023810,0.000000,0.023810,0.000000,0.000000,0.02381,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.023810,0.000000,0.047619,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.023810,0.023810,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,CF Markville,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.00000,0.016667,0.016667,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.083333,0.016667,0.000000,0.066667,0.083333,0.016667,0.050000,0.016667,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.00000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.016667,0.033333,0.016667,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.033333,0.016667,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667
2,First Markham Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.019231,0.000000,0.019231,0.019231,0.000000,0.000000,0.000000,0.192308,0.000000,0.019231,0.000000,0.096154,0.000000,0.000000,0.019231,0.057692,0.000000,0.000000,0.019231,0.000000,0.000000,0.00000,0.000000,0.019231,0.038462,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.019231,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.076923,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.019231,0.038462,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.

#### get the top 15 most common venues in each neighborhood

In [23]:
# define a function that return list of venue categories in descending order of popularity
def return_most_common_venues(row, num_top_venues):
    '''
    
    row: neighborhood,
    num_top_venues: number of venue categories returned.
    
    '''
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
# return a dataframe showing the top 15 most common venues categories
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Mall/plaza']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_ranked = pd.DataFrame(columns = columns)
venues_ranked['Mall/plaza'] = markham_grouped['Mall/plaza']

for ind in np.arange(markham_grouped.shape[0]):
    venues_ranked.iloc[ind, 1:] = return_most_common_venues(markham_grouped.iloc[ind, :], num_top_venues)

venues_ranked

,Mall/plaza,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Alderland Center,Chinese Restaurant,Sandwich Place,Pizza Place,Tea Room,Bubble Tea Shop,Diner,Sporting Goods Shop,Ramen Restaurant,Furniture / Home Store,Shoe Store,Korean Restaurant,Coffee Shop,Food,Grocery Store,Supermarket
1,CF Markville,Chinese Restaurant,Coffee Shop,Restaurant,Clothing Store,Fast Food Restaurant,Cosmetics Shop,Tea Room,Electronics Store,Pizza Place,Toy / Game Store,Grocery Store,Food Court,Kids Store,Jewelry Store,Greek Restaurant
2,First Markham Place,Chinese Restaurant,Coffee Shop,Korean Restaurant,Dessert Shop,Sushi Restaurant,Furniture / Home Store,Thai Restaurant,Sandwich Place,Department Store,Portuguese Restaurant,Halal Restaurant,Climbing Gym,Optical Shop,Yoga Studio,Mexican Restaurant
3,King Square Shopping Mall,Airport,Supermarket,Pizza Place,Field,Hookah Bar,Coffee Shop,Restaurant,Chinese Restaurant,Sporting Goods Shop,Bubble Tea Shop,Greek Restaurant,Gift Shop,Grocery Store,Gym / Fitness Center,Wings Joint
4,Langham Square,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Beer Store,Malay Restaurant,Cosmetics Shop,Fast Food Restaurant,Park,Caribbean Restaurant,Café,Restaurant,Sandwich Place,Seafood Restaurant,Breakfast Spot,Shopping Mall
5,Market Village,Chinese Restaurant,Bakery,Japanese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Tea Room,Snack Place,Pet Store,Miscellaneous Shop,Pizza Place,Park,Optical Shop,Korean Restaurant,Coffee Shop
6,Markham Town Square,Chinese Restaurant,Grocery Store,Bakery,Shopping Mall,Sporting Goods Shop,Liquor Store,Pizza Place,Pet Store,Night Market,Cosmetics Shop,Dessert Shop,Lounge,Diner,Pub,Electronics Store
7,Metro Square,Chinese Restaurant,Bubble Tea Shop,Coffee Shop,Pizza Place,Electronics Store,Supermarket,Tea Room,Sushi Restaurant,Fast Food Restaurant,Korean Restaurant,Sandwich Place,Shoe Store,Furniture / Home Store,Diner,Dessert Shop
8,New Kennedy Square,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Seafood Restaurant,Indian Restaurant,Dessert Shop,Cosmetics Shop,Fast Food Restaurant,New American Restaurant,Park,Caribbean Restaurant,Café,Restaurant,Sandwich Place,Vietnamese Restaurant
9,Pacific Mall,Chinese Restaurant,Bakery,Japanese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Food & Drink Shop,Tea Room,Snack Place,Pet Store,Pizza Place,Gym,Park,Optical Shop,Cosmetics Shop,Miscellaneous Shop


### 3. Clustering using agglomerative clustering

In [26]:
# set number of clusters, best value by observation
kclusters = 5

markham_grouped_clustering = markham_grouped.drop('Mall/plaza', axis = 1)

# run agglomerative clustering
tree = AgglomerativeClustering(n_clusters = kclusters, affinity = 'cosine', linkage = 'complete').fit(markham_grouped_clustering)
#kmeans = KMeans(n_clusters = kclusters, init='k-means++', n_init=10, random_state=0).fit(markham_grouped_clustering)

# add clustering labels to venues_ranked
try:
    venues_ranked.insert(0, 'Cluster Labels', tree.labels_)
except:
    venues_ranked.drop(columns = 'Cluster Labels', inplace = True)
    venues_ranked.insert(0, 'Cluster Labels', tree.labels_)
venues_ranked

,Cluster Labels,Mall/plaza,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,0,Alderland Center,Chinese Restaurant,Sandwich Place,Pizza Place,Tea Room,Bubble Tea Shop,Diner,Sporting Goods Shop,Ramen Restaurant,Furniture / Home Store,Shoe Store,Korean Restaurant,Coffee Shop,Food,Grocery Store,Supermarket
1,2,CF Markville,Chinese Restaurant,Coffee Shop,Restaurant,Clothing Store,Fast Food Restaurant,Cosmetics Shop,Tea Room,Electronics Store,Pizza Place,Toy / Game Store,Grocery Store,Food Court,Kids Store,Jewelry Store,Greek Restaurant
2,0,First Markham Place,Chinese Restaurant,Coffee Shop,Korean Restaurant,Dessert Shop,Sushi Restaurant,Furniture / Home Store,Thai Restaurant,Sandwich Place,Department Store,Portuguese Restaurant,Halal Restaurant,Climbing Gym,Optical Shop,Yoga Studio,Mexican Restaurant
3,3,King Square Shopping Mall,Airport,Supermarket,Pizza Place,Field,Hookah Bar,Coffee Shop,Restaurant,Chinese Restaurant,Sporting Goods Shop,Bubble Tea Shop,Greek Restaurant,Gift Shop,Grocery Store,Gym / Fitness Center,Wings Joint
4,0,Langham Square,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Beer Store,Malay Restaurant,Cosmetics Shop,Fast Food Restaurant,Park,Caribbean Restaurant,Café,Restaurant,Sandwich Place,Seafood Restaurant,Breakfast Spot,Shopping Mall
5,0,Market Village,Chinese Restaurant,Bakery,Japanese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Tea Room,Snack Place,Pet Store,Miscellaneous Shop,Pizza Place,Park,Optical Shop,Korean Restaurant,Coffee Shop
6,4,Markham Town Square,Chinese Restaurant,Grocery Store,Bakery,Shopping Mall,Sporting Goods Shop,Liquor Store,Pizza Place,Pet Store,Night Market,Cosmetics Shop,Dessert Shop,Lounge,Diner,Pub,Electronics Store
7,0,Metro Square,Chinese Restaurant,Bubble Tea Shop,Coffee Shop,Pizza Place,Electronics Store,Supermarket,Tea Room,Sushi Restaurant,Fast Food Restaurant,Korean Restaurant,Sandwich Place,Shoe Store,Furniture / Home Store,Diner,Dessert Shop
8,0,New Kennedy Square,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Seafood Restaurant,Indian Restaurant,Dessert Shop,Cosmetics Shop,Fast Food Restaurant,New American Restaurant,Park,Caribbean Restaurant,Café,Restaurant,Sandwich Place,Vietnamese Restaurant
9,0,Pacific Mall,Chinese Restaurant,Bakery,Japanese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Food & Drink Shop,Tea Room,Snack Place,Pet Store,Pizza Place,Gym,Park,Optical Shop,Cosmetics Shop,Miscellaneous Shop


In [27]:
# drop J-Town
markham_clustered = mall_plaza.set_index('NAME').drop(index = 'J-Town').reset_index().rename(columns = {'NAME': 'Mall/plaza'})

#merge with markham_venues_mod
markham_clustered = markham_clustered.join(venues_ranked.set_index('Mall/plaza'), on='Mall/plaza')
markham_clustered

,Mall/plaza,CITY,ADDRESS,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Alderland Center,Markham,"7102-7158 Warden Ave, Markham, ON",43.824235,-79.326603,0,Chinese Restaurant,Sandwich Place,Pizza Place,Tea Room,Bubble Tea Shop,Diner,Sporting Goods Shop,Ramen Restaurant,Furniture / Home Store,Shoe Store,Korean Restaurant,Coffee Shop,Food,Grocery Store,Supermarket
1,CF Markville,Markham,"5000 Hwy 7, Markham, ON",43.865694,-79.291642,2,Chinese Restaurant,Coffee Shop,Restaurant,Clothing Store,Fast Food Restaurant,Cosmetics Shop,Tea Room,Electronics Store,Pizza Place,Toy / Game Store,Grocery Store,Food Court,Kids Store,Jewelry Store,Greek Restaurant
2,First Markham Place,Markham,"3255 Highway 7 East, Markham, ON",43.851747,-79.348356,0,Chinese Restaurant,Coffee Shop,Korean Restaurant,Dessert Shop,Sushi Restaurant,Furniture / Home Store,Thai Restaurant,Sandwich Place,Department Store,Portuguese Restaurant,Halal Restaurant,Climbing Gym,Optical Shop,Yoga Studio,Mexican Restaurant
3,King Square Shopping Mall,Markham,"9390 Woodbine Ave, Markham, ON",43.872314,-79.365975,3,Airport,Supermarket,Pizza Place,Field,Hookah Bar,Coffee Shop,Restaurant,Chinese Restaurant,Sporting Goods Shop,Bubble Tea Shop,Greek Restaurant,Gift Shop,Grocery Store,Gym / Fitness Center,Wings Joint
4,Langham Square,Markham,"8339 Kennedy Rd, Markham, ON",43.857400,-79.303767,0,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Beer Store,Malay Restaurant,Cosmetics Shop,Fast Food Restaurant,Park,Caribbean Restaurant,Café,Restaurant,Sandwich Place,Seafood Restaurant,Breakfast Spot,Shopping Mall
5,Markham Town Square,Markham,"8601 Warden Ave, Markham, ON",43.857494,-79.331529,4,Chinese Restaurant,Grocery Store,Bakery,Shopping Mall,Sporting Goods Shop,Liquor Store,Pizza Place,Pet Store,Night Market,Cosmetics Shop,Dessert Shop,Lounge,Diner,Pub,Electronics Store
6,Market Village,Markham,"4390 Steeles Avenue East, Markham, ON",43.824568,-79.303737,0,Chinese Restaurant,Bakery,Japanese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Tea Room,Snack Place,Pet Store,Miscellaneous Shop,Pizza Place,Park,Optical Shop,Korean Restaurant,Coffee Shop
7,Metro Square,Markham,"3636 Steeles Ave E, Markham, ON",43.817998,-79.331931,0,Chinese Restaurant,Bubble Tea Shop,Coffee Shop,Pizza Place,Electronics Store,Supermarket,Tea Room,Sushi Restaurant,Fast Food Restaurant,Korean Restaurant,Sandwich Place,Shoe Store,Furniture / Home Store,Diner,Dessert Shop
8,New Kennedy Square,Markham,"8360 Kennedy Rd, Markham, ON",43.859604,-79.302949,0,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Seafood Restaurant,Indian Restaurant,Dessert Shop,Cosmetics Shop,Fast Food Restaurant,New American Restaurant,Park,Caribbean Restaurant,Café,Restaurant,Sandwich Place,Vietnamese Restaurant
9,Pacific Mall,Markham,"4300 Steeles Ave E, Markham, ON",43.825488,-79.306128,0,Chinese Restaurant,Bakery,Japanese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Food & Drink Shop,Tea Room,Snack Place,Pet Store,Pizza Place,Gym,Park,Optical Shop,Cosmetics Shop,Miscellaneous Shop


In [28]:
# create map
map_clusters = folium.Map(location=[latitude_markham, longitude_markham], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.gnuplot(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, neigh, cluster in zip(markham_clustered['LATITUDE'], markham_clustered['LONGITUDE'], markham_clustered['Mall/plaza'], markham_clustered['Cluster Labels']):
    label = folium.Popup(str(neigh) + '\n\n' 'Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster],
        fill = True,
        fill_color = rainbow[cluster],
        fill_opacity = 1.5).add_to(map_clusters)
       
map_clusters

In [ ]:
# sort by cluster label
markham_clustered.sort_values(by = ['Cluster Labels', 'Mall/plaza'], axis = 0, ascending = True, inplace = True)
markham_clustered.reset_index(drop = True, inplace = True)
markham_clustered

We notice that there is no Chinese restaurant at label 1, and it is not very common at label 3. \
Although Chinese restaurant is the most common at label 2 and 4, other cuisines are not common here. \
Therefore, we will only keep label 0 as our final candidates. Label 0 has variety of cuisines, and there are also dessert places. Other venues, such as stores, gyms and parks, are common.

In [ ]:
final_candidates = markham_clustered[markham_clustered['Cluster Labels'] == 0].reset_index(drop = True)
final_candidates

In [ ]:
# remove irrelevant columns
final_candidates = final_candidates[final_candidates.columns.tolist()[0:5]]
final_candidates

#### analyze the Chinese restaurants at these locations

In [ ]:
# get the Chinese restaurants located in our final candidates
chinese_cuisine = markham_venues_mod[markham_venues_mod['Venue Category'] == 'Chinese Restaurant'].reset_index(drop = True)

for place in markham_clustered[markham_clustered['Cluster Labels'] != 0]['Mall/plaza'].tolist():
    chinese_cuisine.drop(index = chinese_cuisine[chinese_cuisine['Mall/plaza'] == place].index, inplace = True)
    chinese_cuisine.reset_index(drop = True, inplace = True)
chinese_cuisine.head()

In [ ]:
# change the venue category back
markham_venues.drop_duplicates(subset = 'Venue ID', inplace = True)
for idx, id in enumerate(chinese_cuisine['Venue ID'].tolist()):
    chinese_cuisine.loc[idx, 'Venue Category'] = markham_venues[markham_venues['Venue ID'] == id]['Venue Category'].values[0]
chinese_cuisine.head()

In [ ]:
# group the cuisine by location and category
chinese_cuisine[['Mall/plaza','Venue', 'Venue Category']].groupby(by = ['Mall/plaza', 'Venue Category']).count()

In [ ]:
#url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format('58c1e6e801f4336e21f31b50', CLIENT_ID, CLIENT_SECRET, VERSION)
#results = requests.get(url).json()
#results

In [ ]:
#results['response']